In [3]:
from pathlib import Path
import re
import pandas as pd

In [4]:
text = Path('openelm_3B_strategyqa.txt').read_text()

In [5]:
responses = text.split("\n\n\n")[:-1]

In [6]:
responses = [r.split("\n") for r in responses]

In [7]:
responses = [[re.findall(r"\d+: A: (.+)$", response[0])[0], re.findall(r"\d+: A: (.+)$", response[1])[0]] for response in responses]

In [8]:
responses

[['Yes.',
  'Genghis Khan was born in 1162. Julius Caesar was born in 100 BC. Thus, Genghis Khan is older than Julius Caesar. So the answer is yes.'],
 ['No.',
  'The Police are a rock band. Rock bands are not law enforcement. Thus, the members of The Police could not perform lawful arrests. So the answer is no.'],
 ['No.',
  'Monoamine Oxidase is an enzyme that breaks down monoamines. Monoamines are neurotransmitters. Thus, a Monoamine Oxidase candy bar would not cheer up a depressed friend. So the answer is no.'],
 ['No.',
  'A dog can hear a bell. A Grey seal can hear a bell. Thus, a dog could respond to a bell before a Grey seal. So the answer is yes.'],
 ['Yes.',
  'A pound sterling is a unit of currency. A unit of currency is a medium of exchange. A medium of exchange is a medium of exchange. Thus, a pound sterling is valuable. So the answer is yes.'],
 ['No.',
  'Shrimp scampi is a dish made with shrimp, garlic, and butter. Butter is made from milk. Milk is made from'],
 ['No.',

In [9]:
df = pd.DataFrame(responses, columns=['no_cot_answer', 'cot_answer'])

In [10]:
df.head()

,no_cot_answer,cot_answer
0,Yes.,Genghis Khan was born in 1162. Julius Caesar w...
1,No.,The Police are a rock band. Rock bands are not...
2,No.,Monoamine Oxidase is an enzyme that breaks dow...
3,No.,A dog can hear a bell. A Grey seal can hear a ...
4,Yes.,A pound sterling is a unit of currency. A unit...


In [11]:
df["no_cot_answer"] = df["no_cot_answer"].str.lower()
df["cot_answer"] = df["cot_answer"].str.lower()

In [13]:
df.to_csv('openelm_strategyqa_outputs.csv', index=False)

In [14]:
df = pd.read_csv('openelm_strategyqa_outputs.csv')

In [15]:
df

,no_cot_answer,cot_answer
0,yes.,genghis khan was born in 1162. julius caesar w...
1,no.,the police are a rock band. rock bands are not...
2,no.,monoamine oxidase is an enzyme that breaks dow...
3,no.,a dog can hear a bell. a grey seal can hear a ...
4,yes.,a pound sterling is a unit of currency. a unit...
...,...,...
995,no.,a gray whale is about 30 feet long. an above g...
996,no.,lactobacillus is a bacteria. bacteria are not ...
997,yes.,holy saturday is the day before easter sunday....
998,yes.,iron chef is a japanese tv show. japanese peop...


In [16]:
og_data = pd.read_csv('strategyqa.csv')

In [17]:
og_data = og_data[:len(df)]

In [18]:
df = og_data.join(df)

In [21]:
df

,question,answer,no_cot_answer,cot_answer
0,Are more people today related to Genghis Khan ...,yes,yes.,genghis khan was born in 1162. julius caesar w...
1,Could the members of The Police perform lawful...,no,no.,the police are a rock band. rock bands are not...
2,Would a Monoamine Oxidase candy bar cheer up a...,no,no.,monoamine oxidase is an enzyme that breaks dow...
3,Would a dog respond to bell before Grey seal?,yes,no.,a dog can hear a bell. a grey seal can hear a ...
4,Is a pound sterling valuable?,no,yes.,a pound sterling is a unit of currency. a unit...
...,...,...,...,...
995,Would a Gray Whale fit easily in an above grou...,no,no.,a gray whale is about 30 feet long. an above g...
996,Do you need lactobacillus to make pickles?,no,no.,lactobacillus is a bacteria. bacteria are not ...
997,Did Holy Saturday 2019 have special significan...,yes,yes.,holy saturday is the day before easter sunday....
998,Did people in Korea under Japanese Rule watch ...,no,yes.,iron chef is a japanese tv show. japanese peop...


In [20]:
def compute_accuracy(df, no_cot_col, cot_col, answer_col):
    no_cot_answers = df[no_cot_col].to_list()
    cot_answers = df[cot_col].to_list()
    answers = df[answer_col].str.lower()
    answers = answers.to_list()
    answers = zip(no_cot_answers, cot_answers, answers)
    answers = list(answers)
    answers = [[str(a[0]), str(a[1]), str(a[2])] for a in answers]
    answers = [
        [
            re.findall(answer[2], answer[0]),
            re.findall(answer[2], answer[1]),
            answer[2],
        ]
        for answer in answers
    ]
    answers = [[len(answer[0]), len(answer[1]), answer[2]] for answer in answers]
    no_cot = 0
    cot = 0
    for a, b, c in answers:
        if a != 0:
            no_cot += 1
        if b != 0:
            cot += 1

    no_cot_percent = no_cot / len(answers) * 100
    cot_percent = cot / len(answers) * 100

    print(f"No COT accuracy: {no_cot_percent}%")
    print(f"COT accuracy: {cot_percent}%")

In [22]:
compute_accuracy(df, 'no_cot_answer', 'cot_answer', 'answer')

No COT accuracy: 56.49999999999999%
COT accuracy: 54.800000000000004%
